# Install & imports

In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: julia_kor. Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd
import numpy as np
import re
from dataclasses import dataclass

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

import transformers
from transformers import (
    AutoTokenizer, 
    RobertaTokenizerFast, 
    DataCollatorForTokenClassification,
    AutoModelForTokenClassification, 
    TrainingArguments, 
    Trainer
)

/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [3]:
import random

def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [4]:
RANDOM_SEED = 1234
set_random_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
%env WANDB_PROJECT=Sentiment_Aspect

env: WANDB_PROJECT=Sentiment_Aspect


# Train

In [6]:
df_train = pd.read_csv('data/aspects_train.csv', index_col=0)
df_val = pd.read_csv('data/aspects_val.csv', index_col=0)

In [7]:
class AspectDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.cat_to_int = {
            "O": 0, 
            "B-Food": 1, 
            "I-Food": 2, 
            "B-Interior": 3,
            "I-Interior": 4, 
            "B-Price": 5,
            "I-Price": 6, 
            "B-Whole": 7, 
            "I-Whole": 8, 
            "B-Service": 9, 
            "I-Service": 10,
            }
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset["tokens"])

    def __getitem__(self, idx):
        tokenized_inputs = self.tokenizer(self.dataset["tokens"].iloc[idx], truncation=True, is_split_into_words=True, max_length=512)
        label = self.dataset["class"].iloc[idx]
        word_ids = tokenized_inputs.word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(self.cat_to_int[label[word_idx]])
            else:
                label_ids.append(self.cat_to_int[label[word_idx]])
            previous_word_idx = word_idx
        tokenized_inputs["labels"] = label_ids
        return tokenized_inputs

In [8]:
MODEL_NAME = 'sberbank-ai/ruRoberta-large'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=11)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you 

In [12]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2).reshape((-1,))
    labels = labels.reshape((-1,))

    true_predictions = [p for (p, l) in zip(predictions, labels) if l != -100]
    true_labels = [l for (p, l) in zip(predictions, labels) if l != -100]

    accuracy = accuracy_score(true_labels, true_predictions)
    return {
        "accuracy": accuracy,
    }

In [13]:
df_train = df_train.groupby("idx").agg(list)
df_val = df_val.groupby("idx").agg(list)

In [14]:
ds_train = AspectDataset(df_train, tokenizer)
ds_dev = AspectDataset(df_val, tokenizer)

In [15]:
for name,param in model.named_parameters():
    if not re.search("classifier|23|22|21|20", name):
        param.requires_grad = False

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to="wandb",
    
    run_name="aspects_cat/experiment_2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=3e-5,
    weight_decay=0.1,
    warmup_ratio=0.06,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 227
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 290
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Thread HandlerThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 51, in run
    self._run()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 102, in _run
    self._process(record)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/internal.py", line 265, in _process
    self._hm.handle(record)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/handler.py", line 132, in handle
    handler(record)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/handler.py", line 142, in handle_request
    handler(record)
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/handler.py", line 672, in handle_request_run_start
    run_meta.probe()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/meta.py", line 230, in probe
    self._setup_git()
  File "/opt/conda/lib/python3.8/site-packages/wandb/sdk/internal/

KeyboardInterrupt: 

Error in callback <function _WandbInit._pause_backend at 0x7f019ca46790> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
int_to_cat = [
    "O", 
    "B-Food", 
    "I-Food", 
    "B-Interior",
    "I-Interior", 
    "B-Price",
    "I-Price", 
    "B-Whole", 
    "I-Whole", 
    "B-Service", 
    "I-Service",
]

def inference(tokens, model, tokenizer):
    model.eval()
    tokenized = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    word_ids = tokenized.word_ids()
    segments_input_ids = []
    segments_attention_mask = []
    segments_word_ids = []
    while len(input_ids) > 512:
        segments_input_ids.append(input_ids[:512])
        segments_attention_mask.append(attention_mask[:512])
        segments_word_ids.append(word_ids[:512])
        input_ids = input_ids[512:]
        attention_mask = attention_mask[512:]
        word_ids = word_ids[512:]
    input_ids_last = torch.zeros(512,)
    input_ids_last[:input_ids.shape[0]] = input_ids
    attention_mask_last = torch.zeros(512,)
    attention_mask_last[:attention_mask.shape[0]] = attention_mask
    segments_input_ids.append(input_ids_last)
    segments_attention_mask.append(attention_mask_last)
    segments_word_ids.append(word_ids + [None] * (512 - len(word_ids)))

    for i, (input_ids, attention_mask, word_ids) in enumerate(zip(segments_input_ids, segments_attention_mask, segments_word_ids)):
        pred = model(input_ids, attention_mask=attention_mask).logits.argmax(dim=2)[0]

        res = []
        prev = None
        for k, j in enumerate(word_ids):
            if j != None and prev != j:
                res.append(int_to_cat[pred[k - i * 512].item()])
            prev = j

In [ ]:
df_test = pd.read_csv('/content/Aspect_Sent_project/data/aspects_test.csv', index_col=0)

In [ ]:
df_test = df_test.groupby("idx").agg(list)

In [ ]:
tokens = df_test.iloc[0]["tokens"]

In [ ]:
model.eval()
tokenized = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
input_ids = tokenized["input_ids"].to(device)
attention_mask = tokenized["attention_mask"].to(device)
word_ids = tokenized.word_ids()

pred = model(input_ids, attention_mask=attention_mask).logits.argmax(dim=2)[0]
prev = None
for k, j in enumerate(word_ids):
    if j != None and prev != j:
        res.append(int_to_cat[pred[k - i * 512].item()])
    prev = j